In [1]:
import requests
import time
import numpy as np
import pandas as pd

from keras.layers import Input, Embedding, Flatten, Dot, Concatenate, Dropout, Dense
from keras.models import Model

/home/raymond/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# First environmement

## 1) Constant agent

In [7]:
user_id = 'Y6EKWA0GK1D0VCTN0RT7'
base_url = "http://52.47.62.31"
url_reset = base_url + "/reset"
url_predict = base_url + "/predict"
params = {"user_id" : user_id}
r = requests.get(url=url_reset, params=params)
data = r.json()

In [10]:
data.keys()

dict_keys(['item_history', 'nb_items', 'nb_users', 'next_item', 'next_user', 'rating_history', 'user_history'])

In [17]:
user_id = 'Y6EKWA0GK1D0VCTN0RT7'
base_url = "http://52.47.62.31"
url_reset = base_url + "/reset"
url_predict = base_url + "/predict"
params = {"user_id" : user_id}
r = requests.get(url=url_reset, params=params)
data = r.json()
nb_users = data["nb_users"]
nb_items = data["nb_items"]
user_history = data["user_history"]
item_history = data["item_history"]
rating_history = data["rating_history"]
next_item = data["next_item"]
next_user = data["next_user"]
prediction = 3
params["predicted_score"] = prediction
nb_samples = 1000
mse, mae = 0, 0

for i in range(nb_samples) : 
    time.sleep(0.5)
    r = requests.get(url=url_predict, params=params)
    d = r.json()
    rating = d["rating"]
    #next_user = d["next_user"]
    #next_item = d["next_item"]
    mse += (rating - prediction) ** 2
    mae += np.abs(rating - prediction)

print("mse : %.4f " % (mse/nb_samples))
print("mae : %.4f " % (mae/nb_samples))

mse : 2.0720 
mae : 1.2380 


In [9]:
user_id = 'Y6EKWA0GK1D0VCTN0RT7'
base_url = "http://52.47.62.31"
url_reset = base_url + "/reset"
url_predict = base_url + "/predict"
params = {"user_id" : user_id}
r = requests.get(url=url_reset, params=params)
data = r.json()
nb_users = data["nb_users"]
nb_items = data["nb_items"]
user_history = data["user_history"]
item_history = data["item_history"]
rating_history = data["rating_history"]
next_user = data["next_user"]
next_item = data["next_item"]
prediction = 3
params["predicted_score"] = prediction
nb_samples = 100
mse, mae = 0, 0

for i in range(nb_samples) : 
    time.sleep(0.5)
    r = requests.get(url=url_predict, params=params)
    d = r.json()
    rating = d["rating"]
    # print(f'user:{next_user}, item:{next_item}, rating:{rating}, prediction:{prediction}')
    next_user = d["next_user"]
    next_item = d["next_item"]
    mse += (rating - prediction) ** 2
    mae += abs(rating - prediction)

print("mse : " + str(mse/nb_samples))
print("mae : " + str(mae/nb_samples))

mse : 1.91
mae : 1.15


## Model with MF using embeddings

In [19]:
user_id = 'Y6EKWA0GK1D0VCTN0RT7'
base_url = "http://52.47.62.31"
url_reset = base_url + "/reset"
url_predict = base_url + "/predict"
params = {"user_id" : user_id}
r = requests.get(url=url_reset, params=params)
data = r.json()
nb_users = data["nb_users"]
nb_items = data["nb_items"]
user_history = data["user_history"]
item_history = data["item_history"]
rating_history = data["rating_history"]
next_user = data["next_user"]
next_item = data["next_item"]
nb_samples = 100
mse, mae = 0, 0
train_ratings = pd.DataFrame({'user_id': user_history, 'item_id': item_history, 'rating': rating_history})

# Model
user_id_input = Input(shape=[1],name='user')
item_id_input = Input(shape=[1], name='item')
embedding_size = 30
user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)
item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)
user_vecs = Flatten()(user_embedding)
item_vecs = Flatten()(item_embedding)
y = Dot(axes=1)([user_vecs, item_vecs])

model = Model(inputs=[user_id_input, item_id_input], outputs=y)
model.compile(optimizer='adam', loss='MSE')
model.fit([train_ratings["user_id"], train_ratings["item_id"]], train_ratings["rating"],
            batch_size=64, epochs=50, validation_split=0.1,
            shuffle=True)

for i in range(nb_samples) : 
    time.sleep(0.5)
    params["predicted_score"] = model.predict([pd.Series(next_user), pd.Series(next_item)])[0, 0]
    prediction = params["predicted_score"]
    r = requests.get(url=url_predict, params=params)
    d = r.json()
    rating = d["rating"]
    # print(f'user:{next_user}, item:{next_item}, rating:{rating}, prediction:{prediction}')
    next_user = d["next_user"]
    next_item = d["next_item"]
    mse += (rating - prediction) ** 2
    mae += abs(rating - prediction)

print("mse : " + str(mse/nb_samples))
print("mae : " + str(mae/nb_samples))

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 0s 25us/step - loss: 10.4574 - val_loss: 9.9268
Epoch 2/50
9000/9000 [==============================] - 0s 11us/step - loss: 10.0823 - val_loss: 8.8445
Epoch 3/50
9000/9000 [==============================] - 0s 11us/step - loss: 7.3948 - val_loss: 4.9191
Epoch 4/50
9000/9000 [==============================] - 0s 14us/step - loss: 3.3850 - val_loss: 1.8986
Epoch 5/50
9000/9000 [==============================] - 0s 12us/step - loss: 1.3112 - val_loss: 0.8682
Epoch 6/50
9000/9000 [==============================] - 0s 12us/step - loss: 0.7200 - val_loss: 0.6226
Epoch 7/50
9000/9000 [==============================] - 0s 11us/step - loss: 0.5881 - val_loss: 0.5697
Epoch 8/50
9000/9000 [==============================] - 0s 11us/step - loss: 0.5581 - val_loss: 0.5550
Epoch 9/50
9000/9000 [==============================] - 0s 11us/step - loss: 0.5501 - val_loss: 0.5509
Epoch 10/50
9000/9000 [

## Model with MF using embeddings and non-linearities

In [28]:
def recommender_embedding(embedding_size) : 
    
    user_id = 'Y6EKWA0GK1D0VCTN0RT7'
    base_url = "http://52.47.62.31"
    url_reset = base_url + "/reset"
    url_predict = base_url + "/predict"
    params = {"user_id" : user_id}
    r = requests.get(url=url_reset, params=params)
    data = r.json()
    nb_users = data["nb_users"]
    nb_items = data["nb_items"]
    user_history = data["user_history"]
    item_history = data["item_history"]
    rating_history = data["rating_history"]
    next_user = data["next_user"]
    next_item = data["next_item"]
    nb_samples = 100
    mse, mae = 0, 0
    train_ratings = pd.DataFrame({'user_id': user_history, 'item_id': item_history, 'rating': rating_history})

    # Model
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')
    embedding_size = embedding_size
    user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)
    item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    input_vecs = Concatenate()([user_vecs, item_vecs])
    input_vecs = Dropout(0.2)(input_vecs)
    x = Dense(64, activation='relu')(input_vecs)
    y = Dense(1)(x)

    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    model.compile(optimizer='adam', loss='MSE')
    model.fit([train_ratings["user_id"], train_ratings["item_id"]], train_ratings["rating"],
            batch_size=64, epochs=50, validation_split=0.1,
            shuffle=True)

    for i in range(nb_samples) : 
        time.sleep(0.5)
        params["predicted_score"] = model.predict([pd.Series(next_user), pd.Series(next_item)])[0, 0]
        prediction = params["predicted_score"]
        r = requests.get(url=url_predict, params=params)
        d = r.json()
        rating = d["rating"]
        # print(f'user:{next_user}, item:{next_item}, rating:{rating}, prediction:{prediction}')
        next_user = d["next_user"]
        next_item = d["next_item"]
        mse += (rating - prediction) ** 2
        mae += abs(rating - prediction)

    print("mse : " + str(mse/nb_samples))
    print("mae : " + str(mae/nb_samples))
    return 

### 1. Embedding_size = 30

In [38]:
recommender_embedding(embedding_size=30)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 1s 111us/step - loss: 4.2695 - val_loss: 0.6870
Epoch 2/50
9000/9000 [==============================] - 0s 19us/step - loss: 0.6432 - val_loss: 0.5936
Epoch 3/50
9000/9000 [==============================] - 0s 20us/step - loss: 0.6094 - val_loss: 0.5664
Epoch 4/50
9000/9000 [==============================] - 0s 21us/step - loss: 0.5811 - val_loss: 0.5430
Epoch 5/50
9000/9000 [==============================] - 0s 22us/step - loss: 0.5536 - val_loss: 0.5399
Epoch 6/50
9000/9000 [==============================] - 0s 24us/step - loss: 0.5179 - val_loss: 0.5155
Epoch 7/50
9000/9000 [==============================] - 0s 22us/step - loss: 0.4989 - val_loss: 0.5105
Epoch 8/50
9000/9000 [==============================] - 0s 22us/step - loss: 0.4876 - val_loss: 0.5180
Epoch 9/50
9000/9000 [==============================] - 0s 23us/step - loss: 0.4780 - val_loss: 0.5058
Epoch 10/50
9000/9000 [=

### 2. Embedding_size = 40

In [39]:
recommender_embedding(embedding_size=40)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 1s 115us/step - loss: 4.0941 - val_loss: 0.7116
Epoch 2/50
9000/9000 [==============================] - 0s 20us/step - loss: 0.7163 - val_loss: 0.5836
Epoch 3/50
9000/9000 [==============================] - 0s 20us/step - loss: 0.6834 - val_loss: 0.5736
Epoch 4/50
9000/9000 [==============================] - 0s 20us/step - loss: 0.6638 - val_loss: 0.5595
Epoch 5/50
9000/9000 [==============================] - 0s 20us/step - loss: 0.6235 - val_loss: 0.5348
Epoch 6/50
9000/9000 [==============================] - 0s 21us/step - loss: 0.5908 - val_loss: 0.5051
Epoch 7/50
9000/9000 [==============================] - 0s 21us/step - loss: 0.5683 - val_loss: 0.4892
Epoch 8/50
9000/9000 [==============================] - 0s 20us/step - loss: 0.5591 - val_loss: 0.4885
Epoch 9/50
9000/9000 [==============================] - 0s 21us/step - loss: 0.5548 - val_loss: 0.4703
Epoch 10/50
9000/9000 [=

### 3. Embedding_size = 50

In [49]:
recommender_embedding(embedding_size=50)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 2s 222us/step - loss: 3.9380 - val_loss: 0.7342
Epoch 2/50
9000/9000 [==============================] - 0s 26us/step - loss: 0.6929 - val_loss: 0.6663
Epoch 3/50
9000/9000 [==============================] - 0s 27us/step - loss: 0.6682 - val_loss: 0.6556
Epoch 4/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.6436 - val_loss: 0.6276
Epoch 5/50
9000/9000 [==============================] - 0s 27us/step - loss: 0.5978 - val_loss: 0.5858
Epoch 6/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.5615 - val_loss: 0.5662
Epoch 7/50
9000/9000 [==============================] - 0s 27us/step - loss: 0.5430 - val_loss: 0.5598
Epoch 8/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.5297 - val_loss: 0.5476
Epoch 9/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.5248 - val_loss: 0.5586
Epoch 10/50
9000/9000 [=

### 4.Embedding_size = 60

In [48]:
recommender_embedding(embedding_size=60)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 2s 205us/step - loss: 3.6324 - val_loss: 0.7319
Epoch 2/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.7025 - val_loss: 0.6693
Epoch 3/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.6669 - val_loss: 0.6234
Epoch 4/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.5968 - val_loss: 0.5503
Epoch 5/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.5440 - val_loss: 0.5351
Epoch 6/50
9000/9000 [==============================] - 0s 36us/step - loss: 0.5355 - val_loss: 0.5280
Epoch 7/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.5224 - val_loss: 0.5261
Epoch 8/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.5169 - val_loss: 0.5226
Epoch 9/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.5154 - val_loss: 0.5342
Epoch 10/50
9000/9000 [=

### 5.Embedding_size = 80

In [47]:
recommender_embedding(embedding_size=80)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 2s 196us/step - loss: 4.0005 - val_loss: 0.6649
Epoch 2/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.6911 - val_loss: 0.6372
Epoch 3/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.6733 - val_loss: 0.6111
Epoch 4/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.6591 - val_loss: 0.5955
Epoch 5/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.6189 - val_loss: 0.5644
Epoch 6/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.5857 - val_loss: 0.5363
Epoch 7/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.5634 - val_loss: 0.5371
Epoch 8/50
9000/9000 [==============================] - 0s 43us/step - loss: 0.5573 - val_loss: 0.5415
Epoch 9/50
9000/9000 [==============================] - 0s 37us/step - loss: 0.5417 - val_loss: 0.5356
Epoch 10/50
9000/9000 [=

### 6.Embedding_size = 100

In [46]:
recommender_embedding(embedding_size=100)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 2s 174us/step - loss: 3.8010 - val_loss: 0.6877
Epoch 2/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.6754 - val_loss: 0.6904
Epoch 3/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.6569 - val_loss: 0.6769
Epoch 4/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.6334 - val_loss: 0.6363
Epoch 5/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.5964 - val_loss: 0.5837
Epoch 6/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.5686 - val_loss: 0.5894
Epoch 7/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.5525 - val_loss: 0.5710
Epoch 8/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.5453 - val_loss: 0.5561
Epoch 9/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.5272 - val_loss: 0.5515
Epoch 10/50
9000/9000 [=

### 7.Embedding_size = 120

In [44]:
recommender_embedding(embedding_size=120)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 1s 157us/step - loss: 3.6992 - val_loss: 0.7527
Epoch 2/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.7556 - val_loss: 0.7435
Epoch 3/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.7400 - val_loss: 0.7329
Epoch 4/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.7100 - val_loss: 0.6956
Epoch 5/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.6660 - val_loss: 0.6900
Epoch 6/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.6340 - val_loss: 0.6526
Epoch 7/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.6159 - val_loss: 0.6386
Epoch 8/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.6073 - val_loss: 0.6447
Epoch 9/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.5997 - val_loss: 0.6412
Epoch 10/50
9000/9000 [=

### 8.Embedding_size = 150

In [45]:
recommender_embedding(embedding_size=150)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 1s 167us/step - loss: 3.1738 - val_loss: 0.7188
Epoch 2/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.7134 - val_loss: 0.6889
Epoch 3/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.6783 - val_loss: 0.6346
Epoch 4/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.5902 - val_loss: 0.5496
Epoch 5/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.5459 - val_loss: 0.5156
Epoch 6/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.5359 - val_loss: 0.5321
Epoch 7/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.5163 - val_loss: 0.5115
Epoch 8/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.5044 - val_loss: 0.5024
Epoch 9/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.4905 - val_loss: 0.4826
Epoch 10/50
9000/9000 [=

Optimal pour embedding_size $\in [50, 100]$

## 3.2) Avec une couche en plus

In [50]:
def recommender_embedding(embedding_size) : 
    
    user_id = 'Y6EKWA0GK1D0VCTN0RT7'
    base_url = "http://52.47.62.31"
    url_reset = base_url + "/reset"
    url_predict = base_url + "/predict"
    params = {"user_id" : user_id}
    r = requests.get(url=url_reset, params=params)
    data = r.json()
    nb_users = data["nb_users"]
    nb_items = data["nb_items"]
    user_history = data["user_history"]
    item_history = data["item_history"]
    rating_history = data["rating_history"]
    next_user = data["next_user"]
    next_item = data["next_item"]
    nb_samples = 100
    mse, mae = 0, 0
    train_ratings = pd.DataFrame({'user_id': user_history, 'item_id': item_history, 'rating': rating_history})

    # Model
    user_id_input = Input(shape=[1],name='user')
    item_id_input = Input(shape=[1], name='item')
    embedding_size = embedding_size
    user_embedding = Embedding(output_dim=embedding_size, input_dim=nb_users + 1,
                           input_length=1, name='user_embedding')(user_id_input)
    item_embedding = Embedding(output_dim=embedding_size, input_dim=nb_items + 1,
                           input_length=1, name='item_embedding')(item_id_input)
    user_vecs = Flatten()(user_embedding)
    item_vecs = Flatten()(item_embedding)
    input_vecs = Concatenate()([user_vecs, item_vecs])
    x = Dense(64, activation='relu')(input_vecs)
    x = Dropout(0.5)(x)
    x = Dense(32, activation='relu')(x)
    y = Dense(1)(x)

    model = Model(inputs=[user_id_input, item_id_input], outputs=y)
    model.compile(optimizer='adam', loss='MSE')
    model.fit([train_ratings["user_id"], train_ratings["item_id"]], train_ratings["rating"],
            batch_size=64, epochs=50, validation_split=0.1,
            shuffle=True)

    for i in range(nb_samples) : 
        time.sleep(0.5)
        params["predicted_score"] = model.predict([pd.Series(next_user), pd.Series(next_item)])[0, 0]
        prediction = params["predicted_score"]
        r = requests.get(url=url_predict, params=params)
        d = r.json()
        rating = d["rating"]
        # print(f'user:{next_user}, item:{next_item}, rating:{rating}, prediction:{prediction}')
        next_user = d["next_user"]
        next_item = d["next_item"]
        mse += (rating - prediction) ** 2
        mae += abs(rating - prediction)

    print("mse : " + str(mse/nb_samples))
    print("mae : " + str(mae/nb_samples))
    return 

In [56]:
recommender_embedding(40)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 3s 284us/step - loss: 3.9986 - val_loss: 0.6473
Epoch 2/50
9000/9000 [==============================] - 0s 25us/step - loss: 0.7713 - val_loss: 0.5788
Epoch 3/50
9000/9000 [==============================] - 0s 25us/step - loss: 0.7229 - val_loss: 0.5488
Epoch 4/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.6727 - val_loss: 0.5429
Epoch 5/50
9000/9000 [==============================] - 0s 27us/step - loss: 0.6528 - val_loss: 0.5661
Epoch 6/50
9000/9000 [==============================] - 0s 27us/step - loss: 0.6324 - val_loss: 0.5161
Epoch 7/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.6096 - val_loss: 0.5135
Epoch 8/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.5868 - val_loss: 0.5149
Epoch 9/50
9000/9000 [==============================] - 0s 25us/step - loss: 0.5784 - val_loss: 0.5151
Epoch 10/50
9000/9000 [=

In [57]:
recommender_embedding(50)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 3s 286us/step - loss: 3.6759 - val_loss: 0.6743
Epoch 2/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.8460 - val_loss: 0.6378
Epoch 3/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.7927 - val_loss: 0.5837
Epoch 4/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.7472 - val_loss: 0.5693
Epoch 5/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.6991 - val_loss: 0.5643
Epoch 6/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.6721 - val_loss: 0.5628
Epoch 7/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.6643 - val_loss: 0.5447
Epoch 8/50
9000/9000 [==============================] - 0s 27us/step - loss: 0.6485 - val_loss: 0.5360
Epoch 9/50
9000/9000 [==============================] - 0s 27us/step - loss: 0.6164 - val_loss: 0.5490
Epoch 10/50
9000/9000 [=

In [58]:
recommender_embedding(60)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 3s 318us/step - loss: 3.2584 - val_loss: 0.8338
Epoch 2/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.8995 - val_loss: 0.7885
Epoch 3/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.7899 - val_loss: 0.7179
Epoch 4/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.7557 - val_loss: 0.7053
Epoch 5/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.7206 - val_loss: 0.6929
Epoch 6/50
9000/9000 [==============================] - 0s 29us/step - loss: 0.6902 - val_loss: 0.6853
Epoch 7/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.6715 - val_loss: 0.6757
Epoch 8/50
9000/9000 [==============================] - 0s 28us/step - loss: 0.6494 - val_loss: 0.6781
Epoch 9/50
9000/9000 [==============================] - 0s 31us/step - loss: 0.6210 - val_loss: 0.6483
Epoch 10/50
9000/9000 [=

In [59]:
recommender_embedding(70)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 3s 351us/step - loss: 3.7612 - val_loss: 0.6583
Epoch 2/50
9000/9000 [==============================] - 0s 35us/step - loss: 0.8538 - val_loss: 0.6117
Epoch 3/50
9000/9000 [==============================] - 0s 37us/step - loss: 0.7658 - val_loss: 0.5670
Epoch 4/50
9000/9000 [==============================] - 0s 32us/step - loss: 0.7232 - val_loss: 0.5368
Epoch 5/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.6900 - val_loss: 0.5566
Epoch 6/50
9000/9000 [==============================] - 0s 37us/step - loss: 0.6586 - val_loss: 0.5186
Epoch 7/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.6444 - val_loss: 0.5080
Epoch 8/50
9000/9000 [==============================] - 0s 30us/step - loss: 0.6189 - val_loss: 0.5080
Epoch 9/50
9000/9000 [==============================] - 0s 33us/step - loss: 0.5984 - val_loss: 0.4936
Epoch 10/50
9000/9000 [=

In [60]:
recommender_embedding(80)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 3s 353us/step - loss: 4.5577 - val_loss: 0.6891
Epoch 2/50
9000/9000 [==============================] - 0s 38us/step - loss: 0.9164 - val_loss: 0.6484
Epoch 3/50
9000/9000 [==============================] - 0s 32us/step - loss: 0.8246 - val_loss: 0.5942
Epoch 4/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.7620 - val_loss: 0.5910
Epoch 5/50
9000/9000 [==============================] - 0s 41us/step - loss: 0.7260 - val_loss: 0.5898
Epoch 6/50
9000/9000 [==============================] - 0s 36us/step - loss: 0.6997 - val_loss: 0.5567
Epoch 7/50
9000/9000 [==============================] - 0s 37us/step - loss: 0.6793 - val_loss: 0.5565
Epoch 8/50
9000/9000 [==============================] - 0s 40us/step - loss: 0.6634 - val_loss: 0.5491
Epoch 9/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.6354 - val_loss: 0.5385
Epoch 10/50
9000/9000 [=

In [61]:
recommender_embedding(100)

Train on 9000 samples, validate on 1000 samples
Epoch 1/50
9000/9000 [==============================] - 3s 350us/step - loss: 3.3852 - val_loss: 0.8050
Epoch 2/50
9000/9000 [==============================] - 0s 37us/step - loss: 0.9452 - val_loss: 0.7447
Epoch 3/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.8549 - val_loss: 0.6840
Epoch 4/50
9000/9000 [==============================] - 0s 35us/step - loss: 0.8167 - val_loss: 0.6594
Epoch 5/50
9000/9000 [==============================] - 0s 35us/step - loss: 0.7792 - val_loss: 0.6540
Epoch 6/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.7581 - val_loss: 0.6479
Epoch 7/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.7335 - val_loss: 0.6408
Epoch 8/50
9000/9000 [==============================] - 0s 34us/step - loss: 0.7107 - val_loss: 0.6254
Epoch 9/50
9000/9000 [==============================] - 0s 35us/step - loss: 0.6983 - val_loss: 0.6325
Epoch 10/50
9000/9000 [=

**Optimal pour embedding_size = 50**